In [21]:
# General modules
import os
import openai
from dotenv import load_dotenv

# Language models
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai.api_key = openai_api_key
client = openai.Client()

chat_model_name = 'gpt-4o-mini'
embed_model_name = "all-MiniLM-L6-v2"
# embedding_model_name = 'sentence-transformers/all-mpnet-base-v2'


# Paths
root_dir = r'C:\Users\fbohm\Desktop\Projects\DataScience\cluster_analysis'
path_input = os.path.join(root_dir, "Data", "2024 Trimester 1.xlsx")

path_db_prepared = os.path.join(root_dir, "Data", "db_prepared.json")
path_db_analysed = os.path.join(root_dir, "Data", "db_analysed.json")
path_db_embedded = os.path.join(root_dir, "Data", "db_embedded.json")
path_db_clustered = os.path.join(root_dir, "Data", "db_clustered.json")
path_db_final = os.path.join(root_dir, "Data", "db_final.json")

In [23]:
# my imports
from helper.utils import *
from helper.data_analysis import *
from helper.prompt_templates import *
from helper.embedding import *
from helper.cluster_analysis import *
from helper.cluster_naming import *


columns_of_interest = [
    "24_[OPTIONAL] Is there anything you'd like to share about the game (good, bad, frustrating, improvement, wishlist, etc.)?", '3_OE_LeastEnjoy'
]

configure_api(client, chat_model_name)

In [24]:
data = load_excel_to_data(path_input)

2024-11-21 17:12:02,234 - INFO - Loading Excel file: C:\Users\fbohm\Desktop\Projects\DataScience\cluster_analysis\Data\2024 Trimester 1.xlsx
2024-11-21 17:12:12,129 - INFO - Removing the first entry of the dataset.
2024-11-21 17:12:12,129 - INFO - Excel data successfully loaded and converted to dictionary.


In [26]:
data_cleaned = clean_json_data(data)

2024-11-21 17:15:53,530 - INFO - Cleaned 0 entries from the dataset.


### Filter and enrich the data
#### - This function checks if the cols of interest contain strings that are longer than 3 words
#### - The removed entries in the logger are all the rows where this condition did NOT hold
#### - All entries where it holds get a new JSON key (column) called "player_response"

In [27]:
# This function checks if the cols of interest contain strings that are longer than 3 words
# The removed entries in the logger are all the rows where this condition did NOT hold
# All entries where it holds get a new JSON key (column) called "player_response"
filtered_data = filter_and_enrich_data(data_cleaned, columns_of_interest)

2024-11-21 17:15:57,267 - INFO - Total entries removed: 2518


In [28]:
len(filtered_data)

4254

### Optional: generate a sample size

In [7]:
sample_size = 1100
seed = 42
sample_data = get_random_sample(filtered_data, sample_size, seed=seed)

In [8]:
# save the sample or the cleaned data as JSON
save_to_json(sample_data, path_db_prepared)
# save_to_json(filtered_data, path_db_prepared)

2024-11-21 11:38:43,434 - INFO - Data successfully saved to C:\Users\fbohm\Desktop\Projects\DataScience\cluster_analysis\Data\db_prepared.json


# Translation


In [9]:
# Define col of interest and the col where the ID is specified
col_of_interest = ["player_response"]
id_col = "Unnamed: 0"

In [10]:
data = read_json(path_db_prepared)

In [11]:
# detect the language of the player response
language = detect_player_language(data, id_col, col_of_interest)

In [12]:
language[0]

{'Unnamed: 0': 1520,
 'Respondent ID': 114587000000,
 'Collector ID': 431039728,
 'Start Date': '2024-04-26 21:01:16',
 'End Date': '2024-04-26 21:09:56',
 'IP Address': '172.97.49.205',
 'Email Address': '',
 'First Name': '',
 'Last Name': '',
 'Custom Data 1': '',
 '1_Rate_Overall Rival Stars Horse Racing': '5 (Strongly Like)',
 '1_Rate_Breeding': '1 (Strongly Dislike)',
 '1_Rate_Training my horses': '1 (Strongly Dislike)',
 '1_Rate_Racing - Story': '3 (Neutral)',
 '1_Rate_Steeplechase': '3 (Neutral)',
 '1_Rate_Cross Country': '3 (Neutral)',
 '1_Rate_Free Roam': '3 (Neutral)',
 '1_Rate_Live Events & Leaderboards (Flat racing & Cross Country)': 2,
 '1_Rate_Completing Story Goals': 4,
 '1_Rate_Foal Caring': '1 (Strongly Dislike)',
 '1_Rate_Pasture': 2,
 '1_Rate_Foaling season event': '5 (Strongly Like)',
 '1_Rate_Steeplechase Stars': '3 (Neutral)',
 '1_Rate_Arabian Days': '1 (Strongly Dislike)',
 '1_Rate_Empowering my horse with skills': 4,
 '1_Rate_Customizing my horse with tack': 4,

#### API calls for translation

In [13]:
translated_data = translate_data(language, id_col, prompt_template_translation, api_settings, columns_of_interest)

2024-11-21 11:39:48,537 - INFO - Translating entry ID 3444 (Language: german)
2024-11-21 11:39:49,528 - INFO - Translating entry ID 1446 (Language: german)
2024-11-21 11:39:50,502 - INFO - Translating entry ID 3686 (Language: spanish)
2024-11-21 11:39:51,516 - INFO - Translating entry ID 3460 (Language: french)
2024-11-21 11:39:55,745 - INFO - Translating entry ID 5527 (Language: german)
2024-11-21 11:39:57,728 - INFO - Translating entry ID 6157 (Language: german)
2024-11-21 11:39:58,670 - INFO - Translating entry ID 6253 (Language: german)
2024-11-21 11:39:59,191 - INFO - Translating entry ID 3492 (Language: german)
2024-11-21 11:39:59,894 - INFO - Translating entry ID 2228 (Language: french)
2024-11-21 11:40:00,376 - INFO - Translating entry ID 1841 (Language: german)
2024-11-21 11:40:01,366 - INFO - Translating entry ID 6004 (Language: spanish)
2024-11-21 11:40:01,975 - INFO - Translating entry ID 3407 (Language: german)
2024-11-21 11:40:02,476 - INFO - Translating entry ID 432 (Lan

In [16]:
# Check out the translations
matching_entry = [entry for entry in translated_data if entry.get(id_col, "unknown") == 5784]
matching_entry


[{'Unnamed: 0': 5784,
  'Respondent ID': 114585000000,
  'Collector ID': 431039728,
  'Start Date': '2024-04-24 17:03:56',
  'End Date': '2024-04-24 17:15:42',
  'IP Address': '91.179.214.141',
  'Email Address': '',
  'First Name': '',
  'Last Name': '',
  'Custom Data 1': '',
  '1_Rate_Overall Rival Stars Horse Racing': '3 (Neutral)',
  '1_Rate_Breeding': 2,
  '1_Rate_Training my horses': '3 (Neutral)',
  '1_Rate_Racing - Story': '3 (Neutral)',
  '1_Rate_Steeplechase': '3 (Neutral)',
  '1_Rate_Cross Country': 2,
  '1_Rate_Free Roam': '3 (Neutral)',
  '1_Rate_Live Events & Leaderboards (Flat racing & Cross Country)': '3 (Neutral)',
  '1_Rate_Completing Story Goals': '3 (Neutral)',
  '1_Rate_Foal Caring': '3 (Neutral)',
  '1_Rate_Pasture': '3 (Neutral)',
  '1_Rate_Foaling season event': '3 (Neutral)',
  '1_Rate_Steeplechase Stars': '3 (Neutral)',
  '1_Rate_Arabian Days': '1 (Strongly Dislike)',
  '1_Rate_Empowering my horse with skills': '3 (Neutral)',
  '1_Rate_Customizing my horse wi

# Topic Extraction and Sentiment Analysis

In [17]:
# Parameters
id_column = "Unnamed: 0"                # Column name for entry IDs
columns_of_interest = ["player_response"]     # Which cols should be analyzed?
batch_size = 50                          # Fail-safe batching. The higher the number, the less often the progress is saved.

prepared_data = read_json(path_db_prepared)

# Run analysis
analyse_data(
    translated_data=prepared_data,
    id_column=id_column,
    output_path=path_db_analysed,
    prompt_template_topic=prompt_template_topic,
    prompt_template_sentiment=prompt_template_sentiment,
    api_settings=api_settings,
    columns_of_interest=columns_of_interest,
    batch_size=batch_size
)

2024-11-21 11:42:02,361 - INFO - No existing progress found. Starting fresh.
2024-11-21 11:42:02,362 - INFO - Tokens used so far: Prompt Tokens: 0, Completion Tokens: 0
2024-11-21 11:42:02,363 - INFO - Extracting topics for entry ID 1520
2024-11-21 11:42:04,007 - INFO - Analyzing sentiment for topic 'Hidden Coats' (Entry ID 1520)
2024-11-21 11:42:04,449 - INFO - Analyzing sentiment for topic 'Breeding Difficulty' (Entry ID 1520)
2024-11-21 11:42:04,845 - INFO - Analyzing sentiment for topic 'Breeding Resources' (Entry ID 1520)
2024-11-21 11:42:05,283 - INFO - Tokens used so far: Prompt Tokens: 1101, Completion Tokens: 89
2024-11-21 11:42:05,283 - INFO - Extracting topics for entry ID 320
2024-11-21 11:42:06,452 - INFO - Analyzing sentiment for topic 'Mobile Version' (Entry ID 320)
2024-11-21 11:42:07,285 - INFO - Analyzing sentiment for topic 'Desktop Version' (Entry ID 320)
2024-11-21 11:42:07,917 - INFO - Tokens used so far: Prompt Tokens: 1986, Completion Tokens: 166
2024-11-21 11:4

# Embedding

In [18]:
batch_size = 50
b_override = False  # Change to True if embeddings should be overwritten
embed_key = "topic"  # Change to "sentence" if you want to embed sentences

# Load the JSON data
data = read_json(path_db_analysed)

# Initialize the embedding model once
embed_model = initialize_embedding_model(model_name=embed_model_name)

# Initialize an empty list for all processed results
processed_results = []

# Process data in batches
for batch_start in range(0, len(data), batch_size):
    batch_end = min(batch_start + batch_size, len(data))
    batch = data[batch_start:batch_end]
    logger.info(f"Processing batch {batch_start // batch_size + 1} ({batch_start} to {batch_end})")
    processed_batch = process_batch(batch, embed_model, b_override, embed_key=embed_key)
    processed_results.extend(processed_batch)  # Collect processed batch results

# Convert all processed results to a DataFrame at once
df_table = json_to_table(processed_results)

# Save the final JSON table
save_df_as_json(df_table, path_db_embedded)
logger.info("Embedding and conversion to table format completed.")


2024-11-21 13:12:48,414 - INFO - Loading embedding model: all-MiniLM-L6-v2
C:\Users\fbohm\Desktop\Projects\DataScience\cluster_analysis\helper\embedding.py:25: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name=model_name))
2024-11-21 13:12:53,907 - INFO - PyTorch version 2.4.0+cu124 available.
2024-11-21 13:12:53,919 - INFO - Polars version 1.12.0 available.
2024-11-21 13:12:53,919 - INFO - Duckdb version 1.1.2 available.
2024-11-21 13:12:55,379 - INFO - Use pytorch device_name: cuda
2024-11-21 13:12:55,379 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2024-11-21 13:12:58,706 - INFO - Processing batch 1

# Cluster Analysis

In [19]:
# Adjustable parameters
dimensionality_methods = ['UMAP', 'PCA', 'tSNE']  # Dimensionality reduction methods
kmeans_clusters = [15, 20, 25, 50]  # Number of clusters for KMeans
kmeans_seed = 42  # Seed for reproducibility
include_2d = True  # Whether to include 2D results
include_3d = True  # Whether to include 3D results
hdbscan_params = {"min_cluster_size": 30, "min_samples": 20, "cluster_selection_epsilon": 0.2}  # HDBSCAN params

# t-SNE specific parameter
perplexity = 15  # Set to a default or user-defined value

# Load data
df_total = load_embedded_data(path_db_embedded)
mat = np.array(df_total['embedding'].tolist())

# Apply dimensionality reduction and clustering
apply_clustering(
    df_total,
    mat,
    dimensionality_methods,
    kmeans_clusters,
    path_db_clustered,
    hdbscan_params=hdbscan_params,
    kmeans_seed=kmeans_seed,
    include_2d=include_2d,
    include_3d=include_3d
)


2024-11-21 13:29:15,107 - INFO - Loading data from C:\Users\fbohm\Desktop\Projects\DataScience\cluster_analysis\Data\db_embedded.json
2024-11-21 13:29:16,964 - INFO - Loaded 2704 valid entries with embeddings.
2024-11-21 13:29:17,016 - INFO - Applying UMAP with 2 components.
C:\Users\fbohm\Documents\Venvironments\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
2024-11-21 13:29:33,133 - INFO - Applying HDBSCAN on UMAP 2D with params: {'min_cluster_size': 30, 'min_samples': 20, 'cluster_selection_epsilon': 0.2}
2024-11-21 13:29:33,195 - INFO - Applying KMeans with 15 clusters on UMAP 2D.
C:\Users\fbohm\Documents\Venvironments\Lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python 

# Cluster Naming

In [20]:
# Paths and parameters
dimensionality_methods = ["UMAP", "PCA", "tSNE"]
clustering_algorithms = ["hdbscan", "kmeans"]
kmeans_clusters = [15, 20, 25, 50]  # Number of clusters for KMeans
max_centers = 8  # Maximum number of topics for naming

# Load data
df_total = load_json_into_df(path_db_clustered)

# Process clusters and generate names
df_total = process_clusters(df_total, dimensionality_methods, clustering_algorithms, kmeans_clusters, max_centers,
                            api_settings)

# Save results
save_data_for_streamlit(df_total, path_db_final)

2024-11-21 14:11:17,505 - INFO - Loading data from C:\Users\fbohm\Desktop\Projects\DataScience\cluster_analysis\Data\db_clustered.json
2024-11-21 14:11:18,345 - INFO - Found 8 Topics for hdbscan_UMAP_2D ID: 18
2024-11-21 14:11:19,253 - INFO - Generated cluster name: Coat Color and Pattern Variations
2024-11-21 14:11:19,253 - INFO -  Tokens used so far: Prompt Tokens: 85, Completion Tokens: 7
2024-11-21 14:11:19,268 - INFO - Found 8 Topics for hdbscan_UMAP_2D ID: 17
2024-11-21 14:11:19,701 - INFO - Generated cluster name: Breeding Strategies and Practices
2024-11-21 14:11:19,701 - INFO -  Tokens used so far: Prompt Tokens: 166, Completion Tokens: 12
2024-11-21 14:11:19,747 - INFO - Found 8 Topics for hdbscan_UMAP_2D ID: 22
2024-11-21 14:11:20,342 - INFO - Generated cluster name: Gameplay and Game Design Insights
2024-11-21 14:11:20,342 - INFO -  Tokens used so far: Prompt Tokens: 242, Completion Tokens: 17
2024-11-21 14:11:20,358 - INFO - Found 8 Topics for hdbscan_UMAP_2D ID: 19
2024-1